In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Embedding, TimeDistributed, RepeatVector, Merge, LSTM
from keras.preprocessing import sequence
import cv2, json, numpy as np

Using TensorFlow backend.


# Training Data Preprocessing

In [3]:
#Load data
with open('annotations/captions_train2014.json') as data:
    data = json.load(data)
captions = data['annotations']
images = data['images']

#5 training examples
train = images
image_ids = [i['id'] for i in train]

In [4]:
#pre-process captions
train_indices = [next(index for (index, d) in enumerate(captions) if d["image_id"] == i) for i in image_ids]

KeyboardInterrupt: 

In [ ]:
train_captions = [captions[i]['caption'] for i in train_indices]

In [ ]:
train_captions = ["START " + i.replace('.', '').lower() + " END" for i in train_captions]

In [ ]:
#get filename for each image with corresponding captions
train_files = ["train2014/" + train[i]['file_name'] for i in range(len(train_captions))]


In [ ]:
#pre-process image
images = []
for image in train_files:
    img = cv2.imread(image)
    img.resize((224,224,3))
    images.append(img)
images = np.asarray(images)

In [ ]:
#build vocabulary and captions
words = [txt.split() for txt in train_captions]
unique = []
for word in words:
    unique.extend(word)
unique = list(set(unique))

In [ ]:
# word_index = {}
# index_word = {}
index_word = {i: word for (i, word) in enumerate(unique)}
word_index = {word: i for (i, word) in enumerate(unique)}
# for i,word in enumerate(unique):
#     word_index[word] = i
#     index_word[i] = word

In [ ]:
captions = [[word_index[txt] for txt in text.split()] for text in train_captions]
# for text in train_captions:
#     one = 
#     captions.append(one)

In [ ]:
max_caption_len = max(len(i) for i in captions)
vocab_size = len(index_word)

In [ ]:
captions = sequence.pad_sequences(captions, maxlen=max_caption_len,padding='post')
next_words = np.zeros((len(train_captions),vocab_size))
for i,text in enumerate(train_captions):
    text = text.split()
    x = [word_index[txt] for txt in text]
    x = np.asarray(x)
    next_words[i,x] = 1

## Load VGG16 Conv Net

In [ ]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPoolinga2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [ ]:
#Initialize pre-trained weights
image_model = VGG_16('vgg16.h5')

In [ ]:
#Remove last 4 layers to get 4096D activations to pass into RNN
image_model.layers.pop()

image_model.summary()

In [ ]:
#Freeze VGG16 weights
for layer in image_model.layers:
    layer.trainable = False

## LSTM captions model

In [ ]:
language_model = Sequential()
language_model.add(Embedding(vocab_size, 256, input_length=max_caption_len))
language_model.add(LSTM(output_dim=128, return_sequences=True))
language_model.add(TimeDistributed(Dense(128)))

In [ ]:
language_model.summary()

## Merge VGG16 and LSTM

In [ ]:
# let's repeat the image vector to turn it into a sequence.
print("Repeat model loading")
image_model.add(RepeatVector(max_caption_len))
print("Repeat model loaded")
# the output of both models will be tensors of shape (samples, max_caption_len, 128).
# let's concatenate these 2 vector sequences.
print("Merging")
model = Sequential()
model.add(Merge([image_model, language_model], mode='concat', concat_axis=-1))
# let's encode this vector sequence into a single vector
model.add(LSTM(256, return_sequences=False))
# which will be used to compute a probability
# distribution over what the next word in the caption should be!
model.add(Dense(vocab_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
print("Merged")

In [ ]:
model.fit([images, captions], next_words, batch_size=1, epochs=5)

In [ ]:
model.save_weights('image_caption_weights.h5')

In [ ]:
captions

In [ ]:
model.summary()

In [ ]:
def test_single(x):
    test = np.expand_dims(x, axis = 0)
    return test

In [ ]:
word_index['START']

In [ ]:
word_index['END']

In [ ]:
max_caption_len

In [ ]:
capt = np.zeros(max_caption_len)
capt[0] = word_index['START']
capt[1] = 12

In [ ]:
capt

In [ ]:
captions

In [ ]:
def predict(image, caption, max_caption_len):
    result = np.zeros(max_caption_len)
    result[0] = word_index['START']
    result[1] = 12
    for i in range(max_caption_len):
        prediction = model.predict_classes([test_single(image), test_single(caption)])
        result[i+2] = prediction
        print(result)
        if prediction == 3:
            break
        if result[max_caption_len - 1] != 0:
            break
    return result

In [ ]:
predict(images[0], capt, max_caption_len)

In [ ]:
capt